# Heart Experiment: 3D Point Cloud Analysis

In this notebook, we will:

1. **Load** multiple PLY point cloud files (in batches of 100 to avoid excessive memory usage).
2. **Create** a Pandas DataFrame for each point cloud.
3. **Compute** a frame-to-frame difference (delta) for each point in each coordinate (`x`, `y`, `z`).
4. **Visualize** the time series of changes.

**Important Note**  
This example assumes that each row (point) in the point cloud at time `t` corresponds to the *same physical point* at time `t-1`. In many real-world scenarios, the number and order of points may differ between consecutive frames. Proper alignment and correspondence would require additional steps (e.g., using nearest neighbors, point cloud registration, or other tracking methods). For this simple demonstration, we proceed under the assumption that row `i` in frame `t` corresponds to row `i` in frame `t-1`.


In [6]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Optional: If you want to use open3d to read ply files:
# !pip install open3d
try:
    import open3d as o3d
    USE_OPEN3D = True
except ImportError:
    USE_OPEN3D = False

# For plotting in Jupyter
%matplotlib inline

**Explanation**:  
- We import `os` and `glob` to help with file paths.  
- We import `numpy` and `pandas` for data handling.  
- We import `matplotlib.pyplot` for plotting.  
- We optionally import `open3d` if available for convenient PLY file reading. Otherwise, we will write a custom PLY reader.


## 2. Define a Function to Read a Single PLY File

In [7]:
def read_ply_as_dataframe(ply_path):
    """
    Reads an ASCII PLY file and returns a DataFrame containing 
    the columns: ['x', 'y', 'z', 'nx', 'ny', 'nz'].
    """
    if USE_OPEN3D:
        # Use open3d for convenience
        pcd = o3d.io.read_point_cloud(ply_path)
        points = np.asarray(pcd.points)
        if pcd.has_normals():
            normals = np.asarray(pcd.normals)
        else:
            normals = np.zeros_like(points)
        data = np.hstack([points, normals])
    else:
        # Fallback: manual parsing of ASCII PLY
        data_lines = []
        with open(ply_path, 'r') as f:
            header_ended = False
            for line in f:
                if header_ended:
                    values = line.strip().split()
                    if len(values) == 6:
                        data_lines.append([float(v) for v in values])
                if line.strip() == "end_header":
                    header_ended = True
        data = np.array(data_lines, dtype=np.float32)
    
    return pd.DataFrame(data, columns=["x", "y", "z", "nx", "ny", "nz"])


**Explanation**:  
- If `open3d` is available, we use `read_point_cloud` to read the file.  
- Otherwise, we manually read the ASCII PLY, skip the header until `end_header`, then parse each line as floats.  
- We then create a Pandas DataFrame with the columns `["x", "y", "z", "nx", "ny", "nz"]`.


## 3. Sliding Window Over Point Cloud Files

In [8]:
def get_ply_file_paths(data_dir):
    """
    Returns a sorted list of .ply files from the given data directory 
    (including nested subdirectories).
    """
    ply_files = glob.glob(os.path.join(data_dir, "**", "*.ply"), recursive=True)
    ply_files.sort()
    return ply_files

def sliding_window_ply_files(ply_files, window_size=100):
    """
    Generator that yields chunks of up to 'window_size' .ply file paths at a time.
    """
    for i in range(0, len(ply_files), window_size):
        yield ply_files[i:i+window_size]


**Explanation**:  
- `get_ply_file_paths` collects all PLY files in the directory (and subdirectories) using `glob` and sorts them.  
- `sliding_window_ply_files` is a generator that splits the list of files into chunks of size 100 (by default). This approach helps avoid loading all files into memory at once.


## 4. Computing Frame-to-Frame Differences

In [9]:
def compute_pointcloud_deltas(ply_file_chunk):
    """
    Given a list of PLY file paths (a chunk),
    read them in order, store each as a DataFrame,
    then compute the difference (delta) between consecutive frames.
    
    Returns:
        frames: list of DataFrames (each containing x,y,z,nx,ny,nz)
        deltas: list of DataFrames (each containing delta_x, delta_y, delta_z, delta_nx, delta_ny, delta_nz)
    """
    frames = []
    deltas = []
    
    prev_df = None
    for idx, ply_path in enumerate(ply_file_chunk):
        df = read_ply_as_dataframe(ply_path)
        frames.append(df)
        
        if prev_df is not None:
            # Ensure both DataFrames have the same number of rows
            if len(df) == len(prev_df):
                delta_df = df[["x", "y", "z", "nx", "ny", "nz"]] - prev_df[["x", "y", "z", "nx", "ny", "nz"]]
                delta_df.columns = ["delta_x", "delta_y", "delta_z", "delta_nx", "delta_ny", "delta_nz"]
                deltas.append(delta_df)
            else:
                print(f"Warning: Different number of points between {ply_file_chunk[idx-1]} and {ply_path}")
                row_count = min(len(df), len(prev_df))
                delta_array = np.full((row_count, 6), np.nan)
                delta_df = pd.DataFrame(delta_array, columns=["delta_x", "delta_y", "delta_z", "delta_nx", "delta_ny", "delta_nz"])
                deltas.append(delta_df)
        prev_df = df
    
    return frames, deltas


**Explanation**:  
1. We loop through each PLY file in the chunk.  
2. We read its contents into a DataFrame.  
3. We subtract the *previous* DataFrame from the *current* DataFrame if the row counts match (computing `delta_x`, `delta_y`, `delta_z`, etc.).  
4. We store both the original frames and the deltas.  

**Note**: If the number of points differ between consecutive frames, we show a warning and (in this example) fill the deltas with `NaN` for the overlapping set of points. You may need a more sophisticated approach depending on your data.


## 5. End-to-End Example

In [ ]:
# Example usage (uncomment and adjust path as needed):
data_directory = "./data"
all_ply_files = get_ply_file_paths(data_directory)

# For demonstration, we'll assume you have enough PLY files to form multiple chunks.

# If you'd like to accumulate ALL deltas (for a specific coordinate) across ALL chunks 
# into a single list and plot them on one line plot, here's how:

# all_delta_x = []  # If you only care about x
# Or for all 6 deltas (delta_x, delta_y, delta_z, delta_nx, delta_ny, delta_nz):
all_deltas = {
    "delta_x": [],
    "delta_y": [],
    "delta_z": [],
    "delta_nx": [],
    "delta_ny": [],
    "delta_nz": []
}

for chunk_idx, ply_chunk in enumerate(sliding_window_ply_files(all_ply_files, window_size=100)):
    frames, deltas = compute_pointcloud_deltas(ply_chunk)
    
    # 'deltas' is a list of DataFrames for consecutive differences between frames in this chunk
    for delta_df in deltas:
        # Accumulate each delta column
        all_deltas["delta_x"].extend(delta_df["delta_x"].values.tolist())
        all_deltas["delta_y"].extend(delta_df["delta_y"].values.tolist())
        all_deltas["delta_z"].extend(delta_df["delta_z"].values.tolist())
        all_deltas["delta_nx"].extend(delta_df["delta_nx"].values.tolist())
        all_deltas["delta_ny"].extend(delta_df["delta_ny"].values.tolist())
        all_deltas["delta_nz"].extend(delta_df["delta_nz"].values.tolist())

# After processing all chunks, we can plot them in one chart (lines only):
plt.figure(figsize=(12,6))
plt.plot(all_deltas["delta_x"], label="Delta X")
plt.plot(all_deltas["delta_y"], label="Delta Y")
plt.plot(all_deltas["delta_z"], label="Delta Z")
plt.plot(all_deltas["delta_nx"], label="Delta NX")
plt.plot(all_deltas["delta_ny"], label="Delta NY")
plt.plot(all_deltas["delta_nz"], label="Delta NZ")
plt.title("All Point Cloud Deltas Across All Chunks")
plt.xlabel("Frame-to-Frame Index (accumulated)")
plt.ylabel("Delta Value")
plt.legend()
plt.show()


**Explanation**:  
- We show how you might put everything together:
  1. Use `get_ply_file_paths` to gather PLY file paths.  
  2. Use `sliding_window_ply_files` with a `window_size=100` to process them in manageable batches.  
  3. For each chunk, run `compute_pointcloud_deltas` to get the DataFrames for frames and their deltas.  
  4. We provide an example of plotting `delta_x` for the *first point* across consecutive frames in the chunk.  
  5. You could similarly look at any point (index) or even compute aggregate statistics (like the mean delta across all points).


# Conclusion

This notebook demonstrates a basic pipeline to:
1. **Load** ASCII-based PLY files (or via `open3d`).
2. **Construct** a Pandas DataFrame for each point cloud.
3. **Compute** simple frame-to-frame deltas (differences) for each coordinate.
4. **Plot** a sample time series of these differences.

In practice, you may need more advanced methods to:
- Align or register consecutive frames (when the number of points or ordering changes).
- Use 3D transformations to track the object of interest (e.g., the beating heart surface).
- Subsample or filter noisy points.

Nevertheless, this approach provides a starting framework for analyzing changes over time in a sequence of PLY point clouds.
